In [11]:
!pip install --upgrade openai


  Using cached openai-1.54.3-py3-none-any.whl.metadata (24 kB)
Using cached openai-1.54.3-py3-none-any.whl (389 kB)
  Attempting uninstall: openai
    Found existing installation: openai 0.28.0
    Uninstalling openai-0.28.0:
      Successfully uninstalled openai-0.28.0


In [12]:
!pip install openai==0.28.0 


  Using cached openai-0.28.0-py3-none-any.whl.metadata (13 kB)
Using cached openai-0.28.0-py3-none-any.whl (76 kB)
  Attempting uninstall: openai
    Found existing installation: openai 1.54.3
    Uninstalling openai-1.54.3:
      Successfully uninstalled openai-1.54.3


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-openai 0.2.3 requires openai<2.0.0,>=1.52.0, but you have openai 0.28.0 which is incompatible.


In [15]:
import os
from dotenv import load_dotenv
import openai
import json
import time

# .env 파일에서 API 키 로드
load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")

# 훈련 데이터를 준비합니다. (data는 .jsonl 형식으로 저장됩니다)
data = [
    {"prompt": "낮의 에스카노르는 어떤 성격인가요?", "completion": "낮의 에스카노르는 오만하고 자신만만하며 힘이 넘칩니다."},
    {"prompt": "밤의 에스카노르는 어떤 성격인가요?", "completion": "밤이 되면 에스카노르는 소심하고 공손하게 말합니다."},
    # 추가 데이터 ...
]

# .jsonl 형식으로 저장
with open("fine_tune_data.jsonl", "w", encoding="utf-8") as f:
    for entry in data:
        json.dump(entry, f, ensure_ascii=False)
        f.write("\n")

# 파일 업로드
with open("fine_tune_data.jsonl", "rb") as file:
    upload_response = openai.File.create(
        file=file,
        purpose="fine-tune"
    )

# 업로드된 파일의 ID 가져오기
file_id = upload_response['id']
print(f"파일이 성공적으로 업로드되었습니다. 파일 ID: {file_id}")

# Fine-Tuning 작업 시작
fine_tune_response = openai.FineTune.create(
    training_file=file_id,
    model="gpt-4"  # gpt-4 모델로 Fine-Tuning
)

# Fine-Tuning 작업 ID 출력
fine_tune_job_id = fine_tune_response['id']
print(f"Fine-tuning 작업이 시작되었습니다. 작업 ID: {fine_tune_job_id}")

# Fine-Tuning 작업 상태 확인 및 완료 대기
status_response = openai.FineTune.retrieve(id=fine_tune_job_id)

# 상태가 완료되기까지 대기
while status_response['status'] != 'succeeded':
    print("Fine-tuning 작업이 진행 중입니다. 상태:", status_response['status'])
    time.sleep(10)  # 10초마다 상태 확인
    status_response = openai.FineTune.retrieve(id=fine_tune_job_id)

# Fine-Tuning 완료 후 추론
fine_tuned_model = status_response['fine_tuned_model']
print(f"Fine-tuning 완료! 새 모델 이름: {fine_tuned_model}")

# Fine-tuned 모델을 사용하여 추론
response = openai.Completion.create(
    model=fine_tuned_model,  # Fine-tuned 모델 이름 사용
    prompt="낮의 에스카노르는 어떤 성격인가요?",
    max_tokens=100
)

print(response['choices'][0]['text'])


파일이 성공적으로 업로드되었습니다. 파일 ID: file-dlgzjWoGatPS4d1n2LTQwaEq


InvalidRequestError: Unknown request URL: POST /v1/fine-tunes. Please check the URL for typos, or see the docs at https://platform.openai.com/docs/api-reference/.